In [169]:
import pandas as pd 
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly as py
import plotly.graph_objs as go
import re
import pickle
%matplotlib inline

## Adding Sources 

In [170]:
nba = pd.read_csv("nba.csv")
nba_extra = pd.read_csv("nba_extra.csv")

In [171]:
nba.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr',
       'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48', 'Unnamed: 24', 'OBPM',
       'DBPM', 'BPM', 'VORP'],
      dtype='object')

In [172]:
nba_extra.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [173]:
nba.head(5)

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,Unnamed: 19,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP
0,1,Alex Abrines\abrinal01,SG,24,OKC,75,1134,9.0,0.567,0.759,...,NaN,1.3,1.0,2.2,0.094,NaN,-0.5,-1.7,-2.2,-0.1
1,2,Quincy Acy\acyqu01,PF,27,BRK,70,1359,8.2,0.525,0.800,...,NaN,-0.1,1.1,1.0,0.036,NaN,-2.0,-0.2,-2.2,-0.1
2,3,Steven Adams\adamsst01,C,24,OKC,76,2487,20.6,0.630,0.003,...,NaN,6.7,3.0,9.7,0.187,NaN,2.2,1.1,3.3,3.3
3,4,Bam Adebayo\adebaba01,C,20,MIA,69,1368,15.7,0.570,0.021,...,NaN,2.3,1.9,4.2,0.148,NaN,-1.6,1.8,0.2,0.8
4,5,Arron Afflalo\afflaar01,SG,32,ORL,53,682,5.8,0.516,0.432,...,NaN,-0.1,0.2,0.1,0.009,NaN,-4.1,-1.8,-5.8,-0.7


In [174]:
nba_extra.head(5)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Alex Abrines\abrinal01,SG,24,OKC,75,8,1134,115,291,...,0.848,26,88,114,28,38,8,25,124,353
1,2,Quincy Acy\acyqu01,PF,27,BRK,70,8,1359,130,365,...,0.817,40,217,257,57,33,29,60,149,411
2,3,Steven Adams\adamsst01,C,24,OKC,76,76,2487,448,712,...,0.559,384,301,685,88,92,78,128,215,1056
3,4,Bam Adebayo\adebaba01,C,20,MIA,69,19,1368,174,340,...,0.721,118,263,381,101,32,41,66,138,477
4,5,Arron Afflalo\afflaar01,SG,32,ORL,53,3,682,65,162,...,0.846,4,62,66,30,4,9,21,56,179


In [175]:
nba_extra.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [176]:
nba.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr',
       'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48', 'Unnamed: 24', 'OBPM',
       'DBPM', 'BPM', 'VORP'],
      dtype='object')

In [177]:
nba_extra['Players'] = nba_extra["Player"].apply(lambda x: x.split("\\")[0])

In [178]:
nba['Players'] = nba["Player"].apply(lambda x: x.split("\\")[0])

In [179]:
nba_extra.head(5)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Players
0,1,Alex Abrines\abrinal01,SG,24,OKC,75,8,1134,115,291,...,26,88,114,28,38,8,25,124,353,Alex Abrines
1,2,Quincy Acy\acyqu01,PF,27,BRK,70,8,1359,130,365,...,40,217,257,57,33,29,60,149,411,Quincy Acy
2,3,Steven Adams\adamsst01,C,24,OKC,76,76,2487,448,712,...,384,301,685,88,92,78,128,215,1056,Steven Adams
3,4,Bam Adebayo\adebaba01,C,20,MIA,69,19,1368,174,340,...,118,263,381,101,32,41,66,138,477,Bam Adebayo
4,5,Arron Afflalo\afflaar01,SG,32,ORL,53,3,682,65,162,...,4,62,66,30,4,9,21,56,179,Arron Afflalo


In [180]:
nba.head(5)

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Players
0,1,Alex Abrines\abrinal01,SG,24,OKC,75,1134,9.0,0.567,0.759,...,1.3,1.0,2.2,0.094,NaN,-0.5,-1.7,-2.2,-0.1,Alex Abrines
1,2,Quincy Acy\acyqu01,PF,27,BRK,70,1359,8.2,0.525,0.800,...,-0.1,1.1,1.0,0.036,NaN,-2.0,-0.2,-2.2,-0.1,Quincy Acy
2,3,Steven Adams\adamsst01,C,24,OKC,76,2487,20.6,0.630,0.003,...,6.7,3.0,9.7,0.187,NaN,2.2,1.1,3.3,3.3,Steven Adams
3,4,Bam Adebayo\adebaba01,C,20,MIA,69,1368,15.7,0.570,0.021,...,2.3,1.9,4.2,0.148,NaN,-1.6,1.8,0.2,0.8,Bam Adebayo
4,5,Arron Afflalo\afflaar01,SG,32,ORL,53,682,5.8,0.516,0.432,...,-0.1,0.2,0.1,0.009,NaN,-4.1,-1.8,-5.8,-0.7,Arron Afflalo


In [181]:
nba_extra_series = nba_extra["Players"].duplicated()

In [182]:
nba_series = nba["Players"].duplicated()

In [183]:
nba.shape

(664, 30)

In [184]:
nba_extra.shape

(664, 31)

## Deleting the duplicates for the NBA players that got traded 

In [185]:
nba_extra_series = nba_extra[nba_extra.duplicated(['Players'], keep=False)]

In [186]:
nba_series = nba[nba_extra.duplicated(['Players'], keep=False)]

In [187]:
nba_extra = nba_extra.drop_duplicates(subset=['Players'], keep='first')

In [188]:
nba = nba.drop_duplicates(subset=['Players'], keep='first')

In [189]:
nba_extra.shape

(540, 31)

In [190]:
nba.shape

(540, 30)

In [191]:
nba_extra=nba_extra[['Rk', 'Players','Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'
       ]]

In [192]:
nba=nba[['Rk', 'Players', 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr',
       'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48', 'Unnamed: 24', 'OBPM',
       'DBPM', 'BPM', 'VORP']]

In [193]:
nba_extra.columns

Index(['Rk', 'Players', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS'],
      dtype='object')

In [194]:
nba_extra.sample(5)

,Rk,Players,Player,Pos,Age,Tm,G,GS,MP,FG,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
21,22,Jamel Artis,Jamel Artis\artisja01,SG,25,ORL,15,1,279,31,...,0.583,4,34,38,18,2,3,8,11,77
592,484,P.J. Tucker,P.J. Tucker\tuckepj01,PF,32,HOU,82,34,2281,172,...,0.717,93,364,457,77,78,26,74,205,502
26,25,Luke Babbitt,Luke Babbitt\babbilu01,SF,28,TOT,50,14,715,91,...,0.773,7,89,96,31,8,7,19,54,259
359,290,Damion Lee,Damion Lee\leeda03,SG,25,ATL,15,11,404,62,...,0.759,11,60,71,29,19,1,14,25,161
512,416,Rodney Purvis,Rodney Purvis\purviro01,SG,23,ORL,16,2,290,35,...,0.867,1,26,27,17,3,3,10,26,96


In [195]:
nba.sample(5)

,Rk,Players,Player,Pos,Age,Tm,G,MP,PER,TS%,...,Unnamed: 19,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP
61,54,Bogdan Bogdanovic,Bogdan Bogdanovic\bogdabo01,SG,25,SAC,78,2175,13.5,0.556,...,NaN,1.8,1.1,2.9,0.064,NaN,0.5,-1.5,-1.0,0.6
54,49,Nemanja Bjelica,Nemanja Bjelica\bjeline01,PF,29,MIN,67,1371,12.6,0.581,...,NaN,1.7,1.1,2.8,0.097,NaN,0.6,-0.1,0.5,0.9
381,310,Josh Magette,Josh Magette\magetjo01,PG,28,ATL,18,216,11.9,0.464,...,NaN,0.3,0.1,0.4,0.092,NaN,-2.3,-3.0,-5.4,-0.2
463,375,David Nwaba,David Nwaba\nwabada01,SG,25,CHI,70,1646,12.2,0.542,...,NaN,1.1,1.4,2.4,0.071,NaN,-2.5,0.9,-1.6,0.1
90,76,Reggie Bullock,Reggie Bullock\bullore01,SF,26,DET,62,1732,13.6,0.615,...,NaN,2.8,1.4,4.2,0.117,NaN,1.3,-0.7,0.7,1.2


In [196]:
nba_extra.drop('Player', axis=1, inplace=True)

In [197]:
nba.drop('Player', axis=1, inplace=True)

In [198]:
nba.drop('Unnamed: 24', axis=1, inplace=True)

In [199]:
nba.drop('Unnamed: 19', axis=1, inplace=True)

In [200]:
nba_extra.columns

Index(['Rk', 'Players', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [201]:
nba.columns

Index(['Rk', 'Players', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr',
       'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP'],
      dtype='object')

In [202]:
nba.rename(columns={'Players':'Player'}, inplace=True)

In [203]:
nba_extra.rename(columns={'Players':'Player'}, inplace=True)

In [204]:
nba.head(5)

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,1,Alex Abrines,SG,24,OKC,75,1134,9.0,0.567,0.759,...,7.4,12.7,1.3,1.0,2.2,0.094,-0.5,-1.7,-2.2,-0.1
1,2,Quincy Acy,PF,27,BRK,70,1359,8.2,0.525,0.800,...,13.3,14.4,-0.1,1.1,1.0,0.036,-2.0,-0.2,-2.2,-0.1
2,3,Steven Adams,C,24,OKC,76,2487,20.6,0.630,0.003,...,13.2,16.7,6.7,3.0,9.7,0.187,2.2,1.1,3.3,3.3
3,4,Bam Adebayo,C,20,MIA,69,1368,15.7,0.570,0.021,...,13.6,15.9,2.3,1.9,4.2,0.148,-1.6,1.8,0.2,0.8
4,5,Arron Afflalo,SG,32,ORL,53,682,5.8,0.516,0.432,...,10.8,12.5,-0.1,0.2,0.1,0.009,-4.1,-1.8,-5.8,-0.7


In [205]:
nba_extra.head(5)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Alex Abrines,SG,24,OKC,75,8,1134,115,291,...,0.848,26,88,114,28,38,8,25,124,353
1,2,Quincy Acy,PF,27,BRK,70,8,1359,130,365,...,0.817,40,217,257,57,33,29,60,149,411
2,3,Steven Adams,C,24,OKC,76,76,2487,448,712,...,0.559,384,301,685,88,92,78,128,215,1056
3,4,Bam Adebayo,C,20,MIA,69,19,1368,174,340,...,0.721,118,263,381,101,32,41,66,138,477
4,5,Arron Afflalo,SG,32,ORL,53,3,682,65,162,...,0.846,4,62,66,30,4,9,21,56,179


In [206]:
nba_extra["Pos"].value_counts()

SG       128
PG       111
C        105
PF       105
SF        88
SF-SG      2
PG-SG      1
Name: Pos, dtype: int64

In [207]:
nba["Pos"].value_counts()

SG       128
PG       111
C        105
PF       105
SF        88
SF-SG      2
PG-SG      1
Name: Pos, dtype: int64

In [208]:
nba_extra[nba_extra['Pos'] == "SF-SG"]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
75,63,Corey Brewer,SF-SG,31,TOT,72,18,1208,142,316,...,0.722,43,111,154,65,79,14,49,139,380
440,358,Shabazz Muhammad,SF-SG,25,TOT,43,2,418,84,188,...,0.780,23,53,76,14,9,3,6,30,214


In [209]:
nba[nba['Pos'] == "SF-SG"]

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
75,63,Corey Brewer,SF-SG,31,TOT,72,1208,11.2,0.534,0.348,...,12.1,14.3,0.7,1.6,2.3,0.091,-1.6,1.0,-0.6,0.4
440,358,Shabazz Muhammad,SF-SG,25,TOT,43,418,15.4,0.510,0.144,...,2.8,23.0,0.5,0.2,0.7,0.081,-1.8,-4.1,-5.9,-0.4


In [210]:
nba_extra.loc[(nba_extra.Pos == 'SF-SG'),'Pos']='SF'

In [211]:
nba.loc[(nba.Pos == 'SF-SG'),'Pos']='SF'

In [212]:
nba_extra[nba_extra['Pos'] == "PG-SG"]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
100,84,Isaiah Canaan,PG-SG,26,TOT,20,1,422,52,137,...,0.902,6,39,45,76,16,1,29,36,172


In [213]:
nba[nba['Pos'] == "PG-SG"]

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
100,84,Isaiah Canaan,PG-SG,26,TOT,20,422,13.5,0.539,0.482,...,15.4,18.9,0.5,0.1,0.6,0.072,0.1,-2.5,-2.4,0.0


In [214]:
nba_extra.loc[(nba_extra.Pos == 'PG-SG'),'Pos']='PG'

In [215]:
nba.loc[(nba.Pos == 'PG-SG'),'Pos']='PG'

In [216]:
nba_extra["Pos"].value_counts()

SG    128
PG    112
C     105
PF    105
SF     90
Name: Pos, dtype: int64

In [217]:
nba["Pos"].value_counts()

SG    128
PG    112
C     105
PF    105
SF     90
Name: Pos, dtype: int64

In [218]:
nba_extra['PPG'] = nba_extra.PTS / nba_extra.G

In [219]:
nba_extra.PPG.sample(2)

591    21.256098
56      7.947368
Name: PPG, dtype: float64

In [220]:
nba_extra['RPG'] = nba_extra.TRB / nba_extra.G
nba_extra['APG'] = nba_extra.AST / nba_extra.G
nba_extra['SPG'] = nba_extra.STL / nba_extra.G
nba_extra['BPG'] = nba_extra.BLK / nba_extra.G
nba_extra['TOPG'] = nba_extra.TOV / nba_extra.G
nba_extra['MPG'] = nba_extra.MP / nba_extra.G
nba_extra['PFG'] = nba_extra.AST / nba_extra.G

In [221]:
nba_extra

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,PF,PTS,PPG,RPG,APG,SPG,BPG,TOPG,MPG,PFG
0,1,Alex Abrines,SG,24,OKC,75,8,1134,115,291,...,124,353,4.706667,1.520000,0.373333,0.506667,0.106667,0.333333,15.120000,0.373333
1,2,Quincy Acy,PF,27,BRK,70,8,1359,130,365,...,149,411,5.871429,3.671429,0.814286,0.471429,0.414286,0.857143,19.414286,0.814286
2,3,Steven Adams,C,24,OKC,76,76,2487,448,712,...,215,1056,13.894737,9.013158,1.157895,1.210526,1.026316,1.684211,32.723684,1.157895
3,4,Bam Adebayo,C,20,MIA,69,19,1368,174,340,...,138,477,6.913043,5.521739,1.463768,0.463768,0.594203,0.956522,19.826087,1.463768
4,5,Arron Afflalo,SG,32,ORL,53,3,682,65,162,...,56,179,3.377358,1.245283,0.566038,0.075472,0.169811,0.396226,12.867925,0.566038
5,6,Cole Aldrich,C,29,MIN,21,0,49,5,15,...,11,12,0.571429,0.714286,0.142857,0.095238,0.047619,0.047619,2.333333,0.142857
6,7,LaMarcus Aldridge,C,32,SAS,75,75,2509,687,1347,...,161,1735,23.133333,8.466667,2.026667,0.573333,1.200000,1.480000,33.453333,2.026667
7,8,Jarrett Allen,C,19,BRK,72,31,1441,234,397,...,147,587,8.152778,5.388889,0.680556,0.388889,1.222222,1.138889,20.013889,0.680556
8,9,Kadeem Allen,PG,25,BOS,18,1,107,6,22,...,15,19,1.055556,0.611111,0.666667,0.166667,0.111111,0.500000,5.944444,0.666667
9,10,Tony Allen,SF,36,NOP,22,0,273,44,91,...,49,103,4.681818,2.136364,0.409091,0.500000,0.136364,0.863636,12.409091,0.409091


In [222]:
nba_extra = nba_extra.round(decimals=2)

In [223]:
nba_extra.dtypes

Rk          int64
Player     object
Pos        object
Age         int64
Tm         object
G           int64
GS          int64
MP          int64
FG          int64
FGA         int64
FG%       float64
3P          int64
3PA         int64
3P%       float64
2P          int64
2PA         int64
2P%       float64
eFG%      float64
FT          int64
FTA         int64
FT%       float64
ORB         int64
DRB         int64
TRB         int64
AST         int64
STL         int64
BLK         int64
TOV         int64
PF          int64
PTS         int64
PPG       float64
RPG       float64
APG       float64
SPG       float64
BPG       float64
TOPG      float64
MPG       float64
PFG       float64
dtype: object

In [224]:
nba

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,1,Alex Abrines,SG,24,OKC,75,1134,9.0,0.567,0.759,...,7.4,12.7,1.3,1.0,2.2,0.094,-0.5,-1.7,-2.2,-0.1
1,2,Quincy Acy,PF,27,BRK,70,1359,8.2,0.525,0.800,...,13.3,14.4,-0.1,1.1,1.0,0.036,-2.0,-0.2,-2.2,-0.1
2,3,Steven Adams,C,24,OKC,76,2487,20.6,0.630,0.003,...,13.2,16.7,6.7,3.0,9.7,0.187,2.2,1.1,3.3,3.3
3,4,Bam Adebayo,C,20,MIA,69,1368,15.7,0.570,0.021,...,13.6,15.9,2.3,1.9,4.2,0.148,-1.6,1.8,0.2,0.8
4,5,Arron Afflalo,SG,32,ORL,53,682,5.8,0.516,0.432,...,10.8,12.5,-0.1,0.2,0.1,0.009,-4.1,-1.8,-5.8,-0.7
5,6,Cole Aldrich,C,29,MIN,21,49,6.0,0.340,0.000,...,5.4,16.8,-0.1,0.1,0.0,-0.013,-7.0,0.1,-6.9,-0.1
6,7,LaMarcus Aldridge,C,32,SAS,75,2509,25.0,0.570,0.068,...,6.8,29.1,7.4,3.5,10.9,0.209,3.0,0.3,3.3,3.3
7,8,Jarrett Allen,C,19,BRK,72,1441,17.5,0.636,0.038,...,15.1,16.3,2.7,1.5,4.2,0.141,-1.3,1.4,0.2,0.8
8,9,Kadeem Allen,PG,25,BOS,18,107,2.6,0.366,0.500,...,25.7,14.6,-0.2,0.1,-0.1,-0.039,-6.7,0.3,-6.4,-0.1
9,10,Tony Allen,SF,36,NOP,22,273,8.7,0.514,0.132,...,15.9,18.9,-0.2,0.2,0.1,0.017,-4.0,-1.3,-5.2,-0.2


In [225]:
nba_extra['PER'] = nba['PER'].values

In [226]:
nba_extra['TS%'] = nba['TS%'].values

In [227]:
nba_extra.Pos.value_counts()

SG    128
PG    112
C     105
PF    105
SF     90
Name: Pos, dtype: int64

In [228]:
nba_extra['pos_num'] = nba_extra.Pos.map({'PG':0, 'SG':1, 'SF':2, 'PF':3, 'C':4})

In [229]:
nba['pos_num'] = nba.Pos.map({'PG':0, 'SG':1, 'SF':2, 'PF':3, 'C':4})

In [230]:
nba.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr',
       'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'pos_num'],
      dtype='object')

In [231]:
nba_extra.rename({'pos_num':'Pos_num'}, axis='columns', inplace=True)

In [232]:
nba.rename({'pos_num':'Pos_num'}, axis='columns', inplace=True)

In [233]:
nba_extra.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PPG',
       'RPG', 'APG', 'SPG', 'BPG', 'TOPG', 'MPG', 'PFG', 'PER', 'TS%',
       'Pos_num'],
      dtype='object')

In [234]:
nba_extra = nba_extra[['Rk', 'Player', 'Pos', 'Pos_num', 'Age', 'Tm', 'G', 'GS', 'MP', 'PPG',
       'RPG', 'APG', 'SPG', 'BPG', 'TOPG', 'MPG', 'PFG', 'PER', 'TS%', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']] 

In [235]:
nba_extra.sample(5)

,Rk,Player,Pos,Pos_num,Age,Tm,G,GS,MP,PPG,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
170,142,Kris Dunn,PG,0,23,CHI,52,43,1525,13.44,...,0.74,24,197,221,310,104,27,150,174,699
211,175,Jonathan Gibson,PG,0,30,BOS,4,0,40,8.50,...,NaN,0,3,3,4,0,0,3,5,34
46,41,Jordan Bell,C,4,23,GSW,57,13,809,4.60,...,0.68,60,147,207,102,35,56,51,92,262
130,108,Quinn Cook,PG,0,24,GSW,33,18,740,9.45,...,0.88,10,72,82,89,12,1,32,59,312
352,283,Jeremy Lamb,SG,1,25,CHO,80,18,1967,12.91,...,0.86,49,275,324,186,61,32,93,138,1033


In [236]:
nba_extra[nba_extra.PPG.notnull()]

,Rk,Player,Pos,Pos_num,Age,Tm,G,GS,MP,PPG,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Alex Abrines,SG,1,24,OKC,75,8,1134,4.71,...,0.85,26,88,114,28,38,8,25,124,353
1,2,Quincy Acy,PF,3,27,BRK,70,8,1359,5.87,...,0.82,40,217,257,57,33,29,60,149,411
2,3,Steven Adams,C,4,24,OKC,76,76,2487,13.89,...,0.56,384,301,685,88,92,78,128,215,1056
3,4,Bam Adebayo,C,4,20,MIA,69,19,1368,6.91,...,0.72,118,263,381,101,32,41,66,138,477
4,5,Arron Afflalo,SG,1,32,ORL,53,3,682,3.38,...,0.85,4,62,66,30,4,9,21,56,179
5,6,Cole Aldrich,C,4,29,MIN,21,0,49,0.57,...,0.33,3,12,15,3,2,1,1,11,12
6,7,LaMarcus Aldridge,C,4,32,SAS,75,75,2509,23.13,...,0.84,246,389,635,152,43,90,111,161,1735
7,8,Jarrett Allen,C,4,19,BRK,72,31,1441,8.15,...,0.78,144,244,388,49,28,88,82,147,587
8,9,Kadeem Allen,PG,0,25,BOS,18,1,107,1.06,...,0.78,4,7,11,12,3,2,9,15,19
9,10,Tony Allen,SF,2,36,NOP,22,0,273,4.68,...,0.52,20,27,47,9,11,3,19,49,103


In [237]:
nba_extra['PPG'] = nba_extra['PPG'].fillna(0.00)
nba_extra['RPG'] = nba_extra['RPG'].fillna(0.00)
nba_extra['APG'] = nba_extra['APG'].fillna(0.00)
nba_extra['SPG'] = nba_extra['SPG'].fillna(0.00)
nba_extra['BPG'] = nba_extra['BPG'].fillna(0.00)
nba_extra['MPG'] = nba_extra['MPG'].fillna(0.00)
nba_extra['TOPG'] = nba_extra['TOPG'].fillna(0.00)
nba_extra['PER'] = nba_extra['PER'].fillna(0.00)
nba_extra['TS%'] = nba_extra['TS%'].fillna(0.00)

In [238]:
# Create feature matrix (X)
feature_cols = ['PPG', 'RPG', 'APG', 'SPG', 'BPG']

In [239]:
# Create feature matrix (X)
X = nba_extra[feature_cols]

In [240]:
# Create response vector (y)
y = nba_extra.Pos_num

In [241]:
# Instantiate with k=5
mymodel = KNeighborsClassifier(n_neighbors=5)

In [242]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X , y, 
                         test_size=0.3, random_state=52 )

In [243]:
# instantiate the estimator
mymodel = KNeighborsClassifier(n_neighbors=5, weights='distance', metric='euclidean')

In [244]:
# fit with training data
mymodel.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='distance')

In [245]:
# predict on the testing data
y_preds = mymodel.predict(X_test)

In [246]:
# evaluate the accuracy score
print(metrics.accuracy_score(y_test, y_preds))

0.4567901234567901


In [247]:
# evaluate other scores
print(metrics.confusion_matrix(y_test, y_preds))

[[16  6  1  4  0]
 [13 21  9  5  3]
 [ 3  9  6  6  2]
 [ 0  5  5  9 13]
 [ 0  1  1  2 22]]


In [248]:
# Add species back in, then reset the index.
train = X_train.copy()
train['Pos_num_trained']=y_train.copy()
train.reset_index(drop=True, inplace=True)

In [249]:
player = [[22, 5, 5, 0, 0]]

In [250]:
mymodel.predict(player)

array([1])

In [251]:
# Pickle the final model for use in the plotly dash app.
file = open('resources/final_model.pkl', 'wb')
pickle.dump(mymodel, file)
file.close()

In [252]:
# Save X-train for visualization purposes
train.to_pickle('resources/training_dataset.pkl')